In [1]:
import pandas as pd
import numpy as np

In [2]:
#download the class movie reviews from here:
#https://docs.google.com/spreadsheets/d/17rCJzmWxqvAu9rkpkgt4ToccIlY4A1Ffuu1W9X3B8Ag/
#then read that in as a pandas dataframe

In [3]:
#downloaded file name -- movie_reviews.csv
M = pd.read_csv('movie_reviews.csv', index_col='Name')

In [4]:
#review dataframe
M.head()

,American Sniper,The Hunger Games: Mockingjay - Part 1,Guardians of the Galaxy,The Lego Movie,The Hobbit,Transformers,Malificent,Big Hero 6,Godzilla,Interstellar,How to Train your Dragon 2,Gone Girl,Divergent,The Fault in Our Stars,Unbroken,300: Rise of an Empire
Name,,,,,,,,,,,,,,,,
Aarti Jaiswal,4.0,NaN,NaN,NaN,4.0,3.0,NaN,5.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
Aditya Dharmasagar,5.0,NaN,NaN,4.0,5.0,4.0,4.0,NaN,4.0,5.0,5.0,5.0,4.0,4.0,NaN,5.0
Adrian Cavallaris,NaN,4.0,3.0,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
Ahmed Muheebuddin,5.0,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aishwarya reddy,NaN,3.0,4.0,3.0,4.0,4.0,3.0,NaN,4.0,4.0,3.0,5.0,NaN,4.0,NaN,3.0


In [5]:
#review shape
M.shape

(104, 16)

In [6]:
def pearson(s1, s2):
    """Take two pd.Series objects and return a pearson correlation."""
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

In [7]:
def get_recs(movie_name, M, num):

    import numpy as np
    reviews = []
    for title in M.columns:
        if title == movie_name:
            continue
        cor = pearson(M[movie_name], M[title])
        if np.isnan(cor):
            continue
        else:
            reviews.append((title, cor))
    
    reviews.sort(key=lambda tup: tup[1], reverse=True)
    return reviews[:num]

    

###Question 1:  What movie is most Similar to 'The fault in our stars' (60 pts)


In [8]:
most_similar = get_recs('The Fault in Our Stars', M, 10)

In [9]:
#the most similar movie would be the first item returned in the array 
most_similar[:1]

[('Divergent', 0.24669725138059095)]

The most similar movie would be Divergent

###Question 2:  Which movie(s) would you most like to see, based on your classmates experience? (40 pts)

In [10]:
#HINTS...

#the movies I've seen
my_ratings = M.ix['Mike'].dropna()
#the movie I've rated the highest
my_ratings.sort(ascending=False)
#the name of the movie I've rated the highest
fav_movie = my_ratings.index[0]

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting


In [12]:
#get Ugo's movies from the list
ugos_ratings = M.ix['Ugo Obialo'].dropna()
ugos_ratings.sort_values(inplace=True, ascending=False)

#Validate
ugos_ratings

Interstellar                  5.0
American Sniper               5.0
How to Train your Dragon 2    4.0
The Hobbit                    4.0
Transformers                  3.0
Guardians of the Galaxy       3.0
Name: Ugo Obialo, dtype: float64

In [39]:
ugos_ratings.index[1]

'American Sniper'

In [13]:
ugos_ratings.size

6

Algorithm for function to determine the movies a user would like to see: 

- Create an array, a, of listed movies and their correlation
- Obtain all the movies that were rated by the user and store in variable, U
- For each movie in U, run the get_recs function to return 5 items. 
- Apppend the 5 items from each run to the array, a 
- After looping through U, sort the movies listed in a, and then output the first 5 or n movies

In [113]:
#credit to lrhache
# https://gist.github.com/lrhache/36a9a5ea5fe7e1f121e3

def list_unique(seq):
    """Remove duplicate from list while keeping order
    Required arguments:
        seq: A list containing all objects
    Returns:
        A list with only unique values. Only the first occurence is kept.
    """
    def filter_uniques(seq):
        # We use set() since it's faster at lookups.
        seen = set()
        for x in seq:
            if x in seen:
                continue
            seen.add(x)
            yield x

    return list(filter_uniques(seq))

In [131]:
def recs_for_user(user_name, M, u_num):
    import numpy as np
    
    #get the movies rated by the user     
    user_movies = M.ix[user_name].dropna()
    user_movies.sort(ascending=False)
    
    
    user_recommendations = []
    
    num_UserMovies = user_movies.size
    
    x = 0
    
    while x < num_UserMovies:
        m_title = user_movies.index[x]
        new_recs = get_recs(m_title, M, 5)
        
        #append each item from the new_recs series to the array
        for idx, (a_name, a_cor) in enumerate(new_recs):
           
            #then, do not append the value if it is in the user's movies
            if a_name in user_movies:
                continue           
                

            #else append a_rec to the list of recommended items
            else:
                user_recommendations.append((a_name, a_cor))
        x += 1
           
    user_recommendations.sort(key=lambda tup: tup[1], reverse=True)
    
    #unzip the user_recommendations to get only the ordered list of movie
    #names 
    m_name, m_cor = zip(*user_recommendations)
    
    #remove duplicates 
    returned_recommendations = list_unique(m_name)
    
    return returned_recommendations[:u_num]
    
    #Validation 
    #return user_recommendations[:u_num]
        

In [132]:
ugos_recs = recs_for_user('Ugo Obialo', M, 5)

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting


In [133]:
#The top 5 recommended movies for Ugo are:
ugos_recs

['The Lego Movie',
 'Godzilla',
 'Gone Girl',
 'Divergent',
 'The Hunger Games: Mockingjay - Part 1']

Question 3: Bonus Question... For all the movies you haven't seen, can you predict how you'd rate them using your the class reviews? (10 pts)

Answer: I would use a K-means cluster to create clusters for movies by a number of common traits/attributes. For example, the gender of the main character, the genre of the movie e.t.c. The ratings/value for  for these clusters would have to be instantiated by each user since they would vary by user. I think the scale of that kind of computation for each user would be the primary challenge. 